In [3]:
import time
import matplotlib.pyplot as plt
import numpy as np 
%matplotlib inline
import pyfvm.mesh as mesh
from pyfvm.field import *
from pyfvm.xnum  import *
from pyfvm.integration import *
import pyfvm.modelphy.euler as euler
import pyfvm.modeldisc      as modeldisc
#
plt.rcParams['figure.dpi']  = 80
plt.rcParams['savefig.dpi'] = 100
#plt.rcParams["animation.html"] = "jshtml"  # for matplotlib 2.1 and above, uses JavaScript
plt.rcParams["animation.html"] = "html5" # for matplotlib 2.0 and below, converts to x264 using ffmpeg video codec

In [4]:
nx   = 200
vavg = 0.
vmag = .01

meshsim  = mesh.unimesh(ncell=nx,  length=10.)

model = euler.model()
bcsym = { 'type': 'sym'}
bcper = { 'type': 'per'}

rhs = modeldisc.fvm(model, meshsim, muscl(vanalbada), bcL=bcsym, bcR=bcsym)
solver = rk3ssp(meshsim, rhs)

# computation
#
nsol    = 120
endtime = 12.
cfl     = .5

def fu(x):
    return vavg + vmag*np.exp(-5*(x-2.)**2)

def fp(x): # gamma = 1.4
    #return 1.
    return (1. + .2*fu(x))**7.  # satisfies C- invariant to make only C+ wave

def frho(x):
    return 1.4 * fp(x)**(1./1.4)

xc    = meshsim.centers()
finit = rhs.fdata_fromprim([ frho(xc), fu(xc), fp(xc) ]) # rho, u, p

fsol = solver.solve(finit, cfl, np.linspace(0., endtime, nsol+1))
solver.show_perf()

In [5]:
# Figure / Plot of final results
varname='velocity'
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(12,4))
ax1.set_ylabel(varname) ; ax1.set_ylim(-vavg-vmag, vavg+vmag)
ax1.grid(linestyle='--', color='0.5')
line1, = fsol[-1].plot(varname, 'k-', axes=ax1)
ax2.set_ylabel('t') ; ax2.set_xlim(0., 10.)
#ax2.grid(linestyle='--', color='0.5')
ttime = [ fsol[i].time for i in range(nsol+1) ]
xx, xt = np.meshgrid(xc, ttime)
solgrid = [ fsol[i].phydata(varname) for i in range(nsol+1) ]
#error = [ np.sqrt(np.sum((fsol[i].phydata(varname)-mach_th)**2)/nx)/mach_th for i in range(nsol+1) ]
#print error
flood  = ax2.contour(xx, xt, solgrid, np.linspace(-vavg-vmag, vavg+vmag, 50))
line2, = ax2.plot([0., 10.], [ttime[-1], ttime[-1]], 'k--')


In [ ]:
import matplotlib.animation as anim
#
def animate(k):
    #i = min(k, nsol)
    fsol[k].set_plotdata(line1, 'mach')
    line2.set_data([0., 10.], [ttime[k], ttime[k]])
    return line1, line2

ani = anim.FuncAnimation(fig=fig, func=animate, frames=range(nsol+1), interval=100, blit=True)
ani
#from IPython.display import HTML
#HTML(ani.to_html5_video()) # if no rcparams